<a href="https://colab.research.google.com/github/jtfreitas/QInfo_Final/blob/main/tensornetwork_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install tensornetwork jax jaxlib -q #--quiet flag
!wget -O final.py https://raw.githubusercontent.com/jtfreitas/QInfo_Final/main/final.py

     |████████████████████████████████| 364 kB 4.1 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
--2022-02-19 19:55:20--  https://raw.githubusercontent.com/jtfreitas/QInfo_Final/main/final.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1719 (1.7K) [text/plain]
Saving to: ‘final.py’

final.py            100%[===================>]   1.68K  --.-KB/s    in 0s      

2022-02-19 19:55:21 (20.2 MB/s) - ‘final.py’ saved [1719/1719]



In [2]:
import tensornetwork as tn
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense
import time
import pandas as pd
from final import MPSLayer
tn.set_default_backend('tensorflow')

#MNIST set

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data('/content/mnist.npz')

y_train = tf.one_hot(y_train, 10)
in_shape = x_train[0].shape

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
def make_vector(pixel):
  return np.array[np.cos(np.pi/2*pixel), np.sin(np.pi/2*pixel)]

def feature_map(data_point):
  feature_map = np.vectorize(make_vector)(data_point)

  return feature_map

#cluster average function, will be used later for generating the new_x, so to train over variations of the data?
def avg_cluster_vect(vector, size):
  shapes = (vector.shape[1]//size, vector.shape[2]//size)
  clustered = np.zeros((vector.shape[0], shapes[0], shapes[1]))
  for k in range(0, vector.shape[0]):
    for i in range(0, shapes[0]):
      for j in range(0, shapes[1]):
        clustered[k,i,j] = np.mean(vector[k,size*i:size*(i+1), size*j:size*(j+1)])
  return clustered
  
# cluster = np.vectorize(avg_cluster)
x_train0 = np.zeros((len(x_train), 14, 14))
for i, sample in enumerate(x_train):
  x_train0[i] = avg_cluster(sample, 2)

x_train = x_train0

IndexError: ignored

In [6]:
def make_vector(pixel):
  return [np.cos(np.pi/2*pixel), np.sin(np.pi/2*pixel)]

def feature_map(data_point):
  feature_map = np.vectorize(make_vector)(data_point)

  return feature_map

trans_x_train = np.array([[make_vector(pixel) for pixel in data_point.flatten()] for data_point in x_train0])

In [7]:
#Building the tensor network

class TNLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(TNLayer, self).__init__()
    # Create the variables for the layer.
    self.a_var = tf.Variable(tf.random.normal(shape=(32,12,10),
                                              stddev=1.0/32.0),
                             name="a", trainable=True)
    self.b_var = tf.Variable(tf.random.normal(shape=(32,336,10),
                                              stddev=1.0/32.0),
                             name="b", trainable=True)
    self.bias = tf.Variable(tf.zeros(shape=(32,32)),
                            name="bias", trainable=True)

  def call(self, inputs):
    # Define the contraction.
    # We break it out so we can parallelize a batch using
    # tf.vectorized_map (see below).
    def f(input_vec, a_var, b_var, bias_var):
      # Reshape to a matrix instead of a vector.
      input_vec = tf.reshape(input_vec, (12,336))

      # Now we create the network.
      a = tn.Node(a_var)
      b = tn.Node(b_var)
      x_node = tn.Node(input_vec)
      a[1] ^ x_node[0]
      b[1] ^ x_node[1]
      a[2] ^ b[2]

      # The TN should now look like this
      #   |     |
      #   a --- b 
      #    \   /
      #      x

      # Now we begin the contraction.
      biggie = a @ x_node
      result = (biggie @ b).tensor

      # Finally, add bias.
      return result + bias_var

    # To deal with a batch of items, we can use the tf.vectorized_map
    # function.
    # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
    result = tf.vectorized_map(
        lambda vec: f(vec, self.a_var, self.b_var, self.bias), inputs)
    return tf.nn.relu(tf.reshape(result, (-1, 1024)))

In [8]:
Dense = tf.keras.layers.Dense
Conv2D = tf.keras.layers.Conv2D
MaxPooling2D = tf.keras.layers.MaxPooling2D
Flatten = tf.keras.layers.Flatten
tn_model = tf.keras.Sequential(
    [
     tf.keras.Input(shape=(*in_shape,1,)),
     Conv2D(28, (4,4), strides = (1,1)),
     MaxPooling2D((2,2)),
     Flatten(),
   #  Dense(1024, activation=tf.nn.relu),
     # Here, we replace the dense layer with our MPS.
     TNLayer(),
     Dense(10, activation='softmax')])
tn_model.summary()

tn_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 28)        476       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 28)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 4032)              0         
                                                                 
 tn_layer (TNLayer)          (None, 1024)              112384    
                                                                 
 dense (Dense)               (None, 10)                10250     
                                                                 
Total params: 123,110
Trainable params: 123,110
Non-trainable params: 0
__________________________________________________

In [9]:
hist_dict = tn_model.fit(x_train, y_train, batch_size = 4096, epochs = 10, verbose = 1, validation_split = 0.2, shuffle=True)

Epoch 1/10
12/12 [==============================] - 84s 7s/step - loss: 2.7231 - accuracy: 0.5800 - val_loss: 0.6625 - val_accuracy: 0.8426
Epoch 2/10
12/12 [==============================] - 93s 8s/step - loss: 0.5877 - accuracy: 0.8618 - val_loss: 0.4144 - val_accuracy: 0.8991
Epoch 3/10
12/12 [==============================] - 85s 7s/step - loss: 0.3638 - accuracy: 0.9079 - val_loss: 0.2927 - val_accuracy: 0.9234
Epoch 4/10
12/12 [==============================] - 83s 7s/step - loss: 0.2638 - accuracy: 0.9270 - val_loss: 0.2347 - val_accuracy: 0.9369
Epoch 5/10
12/12 [==============================] - 82s 7s/step - loss: 0.2108 - accuracy: 0.9385 - val_loss: 0.1971 - val_accuracy: 0.9473
Epoch 6/10
12/12 [==============================] - 83s 7s/step - loss: 0.1764 - accuracy: 0.9485 - val_loss: 0.1763 - val_accuracy: 0.9524
Epoch 7/10
12/12 [==============================] - 96s 8s/step - loss: 0.1523 - accuracy: 0.9553 - val_loss: 0.1620 - val_accuracy: 0.9556
Epoch 8/10
12/12 [==

In [10]:
y_test = tf.one_hot(y_test, 10)

tn_model.evaluate(x_test, y_test)

313/313 [==============================] - 8s 24ms/step - loss: 0.1299 - accuracy: 0.9629


[0.12989506125450134, 0.9628999829292297]

# Evaluating RNA data.

### Data handling

In [11]:
!wget -O RNA_cancer.tar.gz  https://archive.ics.uci.edu/ml/machine-learning-databases/00401/TCGA-PANCAN-HiSeq-801x20531.tar.gz
!tar -xzf RNA_cancer.tar.gz
!mv TCGA-PANCAN-HiSeq-801x20531 RNA_data

--2022-02-19 20:13:45--  https://archive.ics.uci.edu/ml/machine-learning-databases/00401/TCGA-PANCAN-HiSeq-801x20531.tar.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72856320 (69M) [application/x-httpd-php]
Saving to: ‘RNA_cancer.tar.gz’

RNA_cancer.tar.gz   100%[===================>]  69.48M  32.2MB/s    in 2.2s    

2022-02-19 20:13:47 (32.2 MB/s) - ‘RNA_cancer.tar.gz’ saved [72856320/72856320]



In [12]:
#Import data, convert to numpy arrays, perform training split.
#Encode labels as one-hot for using KL-Divergence.

def translate(label, trans_dict):
  """
  Translate string label to unique integer according to dict
  """
  return trans_dict[label]

data = pd.read_csv('/content/RNA_data/data.csv')
labels = pd.read_csv('/content/RNA_data/labels.csv')
X = data[data.keys()[1:]].to_numpy()
translation = {label : i for i, label in enumerate(labels['Class'].unique())}
y_converted = labels['Class'].apply(translate, args=(translation,))
y = y_converted.to_numpy()

#Data has 801 samples. We separate them into 680 -- 121.

X_train, y_train = X[:680], y[:680]
X_test, y_test = X[680:], y[680:]

y_train, y_test = tf.one_hot(y_train, 5), tf.one_hot(y_test, 5)

in_shape = X_train[0].shape


### Classifier definition, training.

#### With tensor network

In [13]:
#Define the tensor network.
#The product of all 2nd indices must equal the input shape.
#In this case, 20531 = 419x7x7, so we use three nodes.
class TNLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(TNLayer, self).__init__()
    # Create the variables for the layer.
    self.a_var = tf.Variable(tf.random.normal(shape=(32,419,5),
                                              stddev=1.0/32.0),
                             name="a", trainable=True)
    self.b_var = tf.Variable(tf.random.normal(shape=(32,7,5,5),
                                              stddev=1.0/32.0),
                             name="b", trainable=True)
    self.c_var = tf.Variable(tf.random.normal(shape=(12,7,5),
                                         stddev=1.0/32.0),
                         name="c", trainable=True)
    self.bias = tf.Variable(tf.zeros(shape=(32,32,12)),
                            name="bias", trainable=True)

  def call(self, inputs):
    # Define the contraction.
    def f(input_vec, a_var, b_var, c_var, bias_var):

      # The TN aspect
      #   |     |     |
      #   a --- b --- c
      #    \    |    /
      #      ---x---

      # Reshape input to tensor.
      input_vec = tf.reshape(input_vec, (419,7,7))


      result = tn.ncon([input_vec, a_var, b_var, c_var], [[1, 2, 3], [-1, 1, 4], [-2, 2, 4, 5], [-3,3,5]])
      #Numbers here are edge numbers. Their position must match the index positions
      # Finally, add bias.
      return result + bias_var

    # To deal with a batch of items, we can use the tf.vectorized_map
    # function.
    # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
    result = tf.vectorized_map(
        lambda vec: f(vec, self.a_var, self.b_var, self.c_var, self.bias), inputs)
    return tf.nn.relu(tf.reshape(result, (-1, 12288))) #perform activation

In [14]:
#Instantiate the model

RNA_tn = tf.keras.Sequential(
    [tf.keras.Input(20531, batch_size=136),
    #MPSLayer(MPS, tf.nn.relu),
    TNLayer(),
    Dense(5, activation='softmax')]
)
RNA_tn.summary()
RNA_tn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tn_layer_1 (TNLayer)        (136, 12288)              85348     
                                                                 
 dense_1 (Dense)             (136, 5)                  61445     
                                                                 
Total params: 146,793
Trainable params: 146,793
Non-trainable params: 0
_________________________________________________________________


In [15]:
start = time.time()
history = RNA_tn.fit(X_train, y_train, batch_size=136, epochs=20, validation_split=0.2, shuffle=True, verbose = 2)
end = time.time()
print(end - start)

Epoch 1/20
4/4 - 4s - loss: 1.5705 - accuracy: 0.3346 - val_loss: 1.3752 - val_accuracy: 0.6985 - 4s/epoch - 1s/step
Epoch 2/20
4/4 - 3s - loss: 1.2857 - accuracy: 0.5037 - val_loss: 1.1396 - val_accuracy: 0.4118 - 3s/epoch - 733ms/step
Epoch 3/20
4/4 - 3s - loss: 1.0313 - accuracy: 0.6710 - val_loss: 0.9008 - val_accuracy: 0.8824 - 3s/epoch - 779ms/step
Epoch 4/20
4/4 - 3s - loss: 0.7697 - accuracy: 0.8713 - val_loss: 0.6099 - val_accuracy: 0.9559 - 3s/epoch - 704ms/step
Epoch 5/20
4/4 - 3s - loss: 0.5009 - accuracy: 0.9816 - val_loss: 0.3728 - val_accuracy: 0.9926 - 3s/epoch - 792ms/step
Epoch 6/20
4/4 - 3s - loss: 0.2781 - accuracy: 0.9926 - val_loss: 0.1881 - val_accuracy: 0.9926 - 3s/epoch - 687ms/step
Epoch 7/20
4/4 - 3s - loss: 0.1385 - accuracy: 0.9963 - val_loss: 0.0968 - val_accuracy: 0.9926 - 3s/epoch - 705ms/step
Epoch 8/20
4/4 - 3s - loss: 0.0674 - accuracy: 0.9982 - val_loss: 0.0514 - val_accuracy: 0.9926 - 3s/epoch - 740ms/step
Epoch 9/20
4/4 - 2s - loss: 0.0349 - accura

In [16]:
RNA_tn.evaluate(X_test, y_test, batch_size=121)

1/1 [==============================] - 1s 632ms/step - loss: 0.0061 - accuracy: 1.0000


[0.006059159990400076, 1.0]

#### With classic dense layers

In [17]:
RNA_classic = tf.keras.Sequential(
    [tf.keras.Input(20531, batch_size=136),
     Dense(1024, activation='relu'),
     Dense(5, activation='softmax')
     ]
)
RNA_classic.summary()
RNA_classic.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (136, 1024)               21024768  
                                                                 
 dense_3 (Dense)             (136, 5)                  5125      
                                                                 
Total params: 21,029,893
Trainable params: 21,029,893
Non-trainable params: 0
_________________________________________________________________


In [18]:
start = time.time()
hist_classic = RNA_classic.fit(X_train, y_train, batch_size = 136, epochs=20, validation_split = 0.2, shuffle=True, verbose=2)
end = time.time()
print(end - start)

Epoch 1/20
4/4 - 2s - loss: 513.8262 - accuracy: 0.2316 - val_loss: 1062.5658 - val_accuracy: 0.1838 - 2s/epoch - 492ms/step
Epoch 2/20
4/4 - 2s - loss: 656.8276 - accuracy: 0.3824 - val_loss: 493.6488 - val_accuracy: 0.3897 - 2s/epoch - 385ms/step
Epoch 3/20
4/4 - 2s - loss: 420.9231 - accuracy: 0.2629 - val_loss: 283.0619 - val_accuracy: 0.3676 - 2s/epoch - 434ms/step
Epoch 4/20
4/4 - 2s - loss: 182.8158 - accuracy: 0.4871 - val_loss: 170.1289 - val_accuracy: 0.3015 - 2s/epoch - 406ms/step
Epoch 5/20
4/4 - 2s - loss: 87.7281 - accuracy: 0.5349 - val_loss: 65.5223 - val_accuracy: 0.3824 - 2s/epoch - 475ms/step
Epoch 6/20
4/4 - 2s - loss: 36.2236 - accuracy: 0.6250 - val_loss: 53.9928 - val_accuracy: 0.4412 - 2s/epoch - 425ms/step
Epoch 7/20
4/4 - 2s - loss: 24.4937 - accuracy: 0.6691 - val_loss: 0.3563 - val_accuracy: 0.9853 - 2s/epoch - 433ms/step
Epoch 8/20
4/4 - 2s - loss: 10.2386 - accuracy: 0.8346 - val_loss: 17.3992 - val_accuracy: 0.8529 - 2s/epoch - 449ms/step
Epoch 9/20
4/4 -

In [19]:
RNA_classic.evaluate(X_test, y_test, batch_size=121)

1/1 [==============================] - 0s 247ms/step - loss: 0.0106 - accuracy: 0.9917


[0.010647586546838284, 0.9917355179786682]